In [205]:
from surprise import accuracy, Dataset, Reader
from surprise.model_selection import KFold, cross_validate, train_test_split
from surprise.prediction_algorithms import KNNBasic, SVD, BaselineOnly, NormalPredictor, NMF, SVDpp
import pandas as pd
import numpy as np

In [206]:
df = pd.read_csv("reviews_cleaned.csv")
df['rate'] = df['rate'] * 2
reviewed_courses = df["course_id"].unique()

In [207]:
courses = pd.read_csv("Course_info.csv")
courses['avg_rating'] = courses['avg_rating'] * 2

In [208]:
reader = Reader(rating_scale=(1, 10))
surprise_data = Dataset.load_from_df(df[["user_id", "course_id", "rate"]], reader)
cross_validate(NormalPredictor(), surprise_data, cv=3)

{'test_rmse': array([1.75715752, 1.74478876, 1.75045948]),
 'test_mae': array([1.21349369, 1.20515721, 1.21438454]),
 'fit_time': (0.11500000953674316, 0.15103387832641602, 0.15703082084655762),
 'test_time': (0.16396760940551758, 0.17497014999389648, 0.17299628257751465)}

In [209]:
trainset = surprise_data.build_full_trainset()
algo = NormalPredictor()
algo.fit(trainset)

In [210]:
def random_user(df):
    users = df['user_id'].unique()
    index = np.random.choice(users.shape[0], 1, replace=False)  
    return users[index][0]

def get_recommendations(user, courses, algo, n):
    predictions = list()
    for course in courses:
        pred = algo.predict(user, course)
        predictions.append([pred.iid, pred.r_ui, pred.est])
    predictions = np.array(predictions)
    return predictions[predictions[:,2].argsort()][::-1][:5]

In [211]:
np.set_printoptions(suppress=True)
user = random_user(df)
recom = get_recommendations(user, reviewed_courses, algo, 5)
recom_id = recom[:,0]
recom

array([[802946, None, 10],
       [1030316, None, 10],
       [536706, None, 10],
       [1276020, None, 10],
       [1467412, None, 10]], dtype=object)

In [212]:
user_courses = list(df[df["user_id"] == user]["course_id"])

In [213]:
courses[courses["id"].isin(user_courses)]

,id,title,is_paid,price,headline,num_subscribers,avg_rating,num_reviews,num_comments,num_lectures,content_length_min,published_time,last_update_date,category,subcategory,topic,language,course_url,instructor_name,instructor_url
19509,922484.0,The Complete Node.js Developer Course (3rd Edi...,True,84.99,Learn Node.js by building real-world applicati...,282270.0,9.280741,67567.0,11451.0,191.0,2103.0,2016-10-04T15:27:39Z,2021-03-22,Development,Programming Languages,Node.Js,English,/course/the-complete-nodejs-developer-course-2/,Andrew Mead,/user/andrewmead/
39113,1565838.0,The Complete 2022 Web Development Bootcamp,True,199.99,Become a Full-Stack Web Developer with just ON...,771176.0,9.396178,228108.0,27723.0,531.0,3939.0,2018-10-25T21:34:15Z,2022-05-09,Development,Web Development,Web Development,English,/course/the-complete-web-development-bootcamp/,Dr. Angela Yu,/user/4b4368a3-b5c8-4529-aa65-2056ec31f37e/
62301,2259120.0,The Complete 2021 Flutter Development Bootcamp...,True,189.99,Officially created in collaboration with the G...,154430.0,9.233921,45169.0,7624.0,233.0,1727.0,2019-05-01T17:03:52Z,2021-12-20,Development,Mobile Development,Google Flutter,English,/course/flutter-bootcamp-with-dart/,Dr. Angela Yu,/user/4b4368a3-b5c8-4529-aa65-2056ec31f37e/


In [214]:
courses[courses["id"].isin(recom_id)]

,id,title,is_paid,price,headline,num_subscribers,avg_rating,num_reviews,num_comments,num_lectures,content_length_min,published_time,last_update_date,category,subcategory,topic,language,course_url,instructor_name,instructor_url
10027,536706.0,"Tableau for Beginners: Get DA Certified, Grow ...",True,39.99,Jump into data visualization with hands-on exe...,66724.0,9.166667,15468.0,2182.0,130.0,389.0,2015-11-09T19:03:20Z,2022-08-26,Business,Business Analytics & Intelligence,Tableau,English,/course/tableau-accelerate-your-career-and-get...,Lukas Halim,/user/lukashalim/
16535,802946.0,Aprende Programación en Java (de Básico a Avan...,True,19.99,En este curso Aprenderás a programar en el len...,15478.0,9.371901,6117.0,1523.0,355.0,3345.0,2017-08-15T19:02:07Z,2018-10-16,Development,Programming Languages,Java,Spanish,/course/aprende-programacion-en-java-desde-cero/,Alejandro Miguel Taboada Sanchez,NaN
22228,1030316.0,Business Model Innovation: Differentiate & Gro...,True,189.99,Use these simple innovation strategies for you...,47263.0,9.153040,15835.0,2669.0,70.0,252.0,2017-04-20T16:18:56Z,2022-09-22,Business,Business Strategy,Innovation,English,/course/disruptive-innovation-business-model-s...,Paul Nekh. | Business | Innovation | Video,/user/pavel-28/
28848,1276020.0,Adobe Photoshop 2018 Completo - do Iniciante a...,True,529.90,Aprenda a tratar fotografias para deixá-las co...,33271.0,9.266666,7637.0,1494.0,207.0,1112.0,2017-07-03T01:36:55Z,2021-07-24,Design,Graphic Design & Illustration,Photoshop,Portuguese,/course/photoshop-para-iniciante-o-basico/,Thiago Christo,/user/thiago-christo/
35697,1467412.0,Crea sistemas POS Inventarios y ventas con PHP...,True,94.99,Aprende a crear sistemas para puntos de venta ...,11463.0,9.321428,3606.0,1072.0,124.0,1608.0,2018-01-02T18:47:09Z,2021-11-08,Development,Web Development,PHP MVC,Spanish,/course/crea-sistemas-pos-inventarios-y-ventas...,Juan Fernando Urrego,/user/juanunativa/
